In [1]:
from pathlib import Path
import pandas as pd

base = Path("../processed")

# carregar os arquivos limpos
geral = pd.read_csv(base / "geral_limpo.csv")
leads = pd.read_csv(base / "leads_limpo.csv")
visitas = pd.read_csv(base / "visitas_limpo.csv")
contratos = pd.read_csv(base / "contratos_limpo.csv")




In [2]:
# ===============================================================
# CRIAÇÃO DE MÉTRICAS E DF_FUNIL
# ===============================================================

# 1. Garantia de Datas (Segurança extra)
date_cols_funil = ["data_criacao", "data_venda", "data_perda"]
for col in date_cols_funil:
    leads[col] = pd.to_datetime(leads[col], errors="coerce")

# -----------------------------
# 2. Criação de Coluna de Safra 
# -----------------------------
#  Permite agrupar leads por mês para comparar com visitas e metas
leads["mes_safra"] = leads["data_criacao"].dt.to_period("M").astype(str)

# -----------------------------
# 3. Flags e Métricas
# -----------------------------
leads["flag_indicado"] = leads["indicado"].astype(int)
leads["flag_venda"] = leads["data_venda"].notna().astype(int)

# Métricas de ciclo (Dias)
leads["dias_ate_venda"] = (leads["data_venda"] - leads["data_criacao"]).dt.days
leads["dias_ate_perda"] = (leads["data_perda"] - leads["data_criacao"]).dt.days

# -----------------------------
# Selecionei apenas as colunas úteis para análise, deixando o DF leve
colunas_analise = [
    "lead_id", "data_criacao", "mes_safra", # Temporais
    "data_venda", "data_perda",
    "utm_source", "sdr_responsavel", "closer_responsavel", # Dimensões
    "profissao", "tipo_lead", "motivo_perda", "indicado",
    "status_final", "flag_venda", "flag_indicado", # Status e Flags
    "dias_ate_venda", "dias_ate_perda" # Métricas
]

# Cria o df_funil oficial
df_funil = leads[colunas_analise].copy()

# Preview para conferência
print(f"Funil gerado com sucesso! Total de linhas: {len(df_funil)}")
df_funil.head()

Funil gerado com sucesso! Total de linhas: 52098


,lead_id,data_criacao,mes_safra,data_venda,data_perda,utm_source,sdr_responsavel,closer_responsavel,profissao,tipo_lead,motivo_perda,indicado,status_final,flag_venda,flag_indicado,dias_ate_venda,dias_ate_perda
0,79342,2024-04-01,2024-04,2024-04-18,2024-07-24,typeform-gentileza,none,raquel,psicologia,qualificado,none,True,VENDIDO,1,1,17.0,114.0
1,79305,2024-04-01,2024-04,2024-04-04,2024-09-11,desconhecido,none,raquel,nutrição,qualificado,none,True,VENDIDO,1,1,3.0,163.0
2,79271,2024-04-01,2024-04,2024-04-02,2024-07-26,instagram,none,bianca,psicologia,qualificado,none,True,VENDIDO,1,1,1.0,116.0
3,79248,2024-04-01,2024-04,2024-04-01,2024-12-06,google,none,bianca,odontologia,qualificado,none,True,VENDIDO,1,1,0.0,249.0
4,79226,2024-04-01,2024-04,2024-04-30,2024-11-25,typeform-blip,none,adriele,outros,qualificado,none,True,VENDIDO,1,1,29.0,238.0


In [3]:

base = Path("../processed")
geral = pd.read_csv(base / "geral_limpo.csv")


# linha de meta (primeira ocorrência de "Vendas")
linha_meta = geral[geral["metrica"] == "Vendas"].iloc[0]

# linha de realizado (segunda ocorrência de "Vendas")
linha_real = geral[geral["metrica"] == "Vendas"].iloc[1]

# ---------------------------
# 3. Selecionar período (Abr/24 → Mar/25)
# ---------------------------
colunas_periodo = [
    col for col in geral.columns 
    if "2024-" in col or "2025-" in col
]

# usar de Abril/24 até Março/25 (12 meses)
colunas_periodo = [c for c in colunas_periodo if "2024-04" <= c <= "2025-03"]

print("Meses usados no cálculo →", colunas_periodo)

# ---------------------------
# 4. Converter para série numérica
# ---------------------------
meta = linha_meta[colunas_periodo].astype(float)
realizado = linha_real[colunas_periodo].astype(float)

# ---------------------------
# 5. Cálculos
# ---------------------------
realizado_total = realizado.sum()
meta_total = meta.sum()
atingimento_medio = realizado_total / meta_total

# ---------------------------
# 6. Exibir diagnóstico
# ---------------------------
print(f"--- Diagnóstico: Atingimento Médio da Meta de Vendas ---")
print(f"Período: Abr/24 a Mar/25 ({len(colunas_periodo)} meses)")
print(f"Realizado Total: {realizado_total:,.0f} vendas")
print(f"Meta Total: {meta_total:,.0f} vendas")
print(f"Atingimento Médio da Meta: {atingimento_medio:.1%}")


Meses usados no cálculo → ['2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12', '2025-01', '2025-02', '2025-03']
--- Diagnóstico: Atingimento Médio da Meta de Vendas ---
Período: Abr/24 a Mar/25 (12 meses)
Realizado Total: 6,084 vendas
Meta Total: 7,285 vendas
Atingimento Médio da Meta: 83.5%


PERGUNTA 02: Qual coluna da base mostra se o lead foi indicado por um membro da Livance?



A coluna da base que mostra se o foi indicado por um membro da Livance é a coluna: "indicado" 

PERGUNTA 03: Qual a taxa de conversão geral em vendas dos leads da Livance?




In [4]:
# --- Análise 3: Taxa de Conversão Geral em Venda ---

# 1. Contar o Total de Leads (entrada do funil)
# Utiliza a contagem única de IDs no dataframe inteiro
total_leads = df_funil['lead_id'].nunique()

# 2. Contar o Total de Leads Vendidos
# Utiliza a flag_venda (onde 1 = VENDIDO) para contar leads únicos
total_vendas = df_funil[df_funil['flag_venda'] == 1]['lead_id'].nunique()

# 3. Calcular a Taxa de Conversão
# Fórmula: (Total de Vendas / Total de Leads)
taxa_conversao_geral = total_vendas / total_leads

# 4. Exibir o resultado formatado
print(f"--- Diagnóstico: Taxa de Conversão Geral ---")
print(f"Total de Leads (Volume de entrada): {total_leads:,.0f}")
print(f"Total de Vendas (Output final): {total_vendas:,.0f}")
print(f"Taxa de Conversão Geral: {taxa_conversao_geral:.2%}")

--- Diagnóstico: Taxa de Conversão Geral ---
Total de Leads (Volume de entrada): 52,098
Total de Vendas (Output final): 6,207
Taxa de Conversão Geral: 11.91%


PERGUNTA 04: Qual canal de aquisição gerou o maior volume de leads?

In [5]:
from IPython.display import display

# --- Análise 4: Ranking de Volume de Leads por Canal de Aquisição (utm_source) ---

# 1. Agrupar por 'utm_source' e contar o total de leads (IDs únicos)
df_volume_canal = df_funil.groupby('utm_source')['lead_id'].nunique().reset_index()

# 2. Renomear a coluna e ordenar pelo volume em ordem decrescente
df_volume_canal.columns = ['canal_aquisicao', 'total_leads']
df_volume_canal = df_volume_canal.sort_values(by='total_leads', ascending=False).reset_index(drop=True)

# 3. Calcular a participação de cada canal no volume total
total_geral = df_volume_canal['total_leads'].sum()
df_volume_canal['participacao_%'] = df_volume_canal['total_leads'] / total_geral

# 4. Formatar as colunas e exibir o resultado (Top 10 canais)
df_exibicao = df_volume_canal.head(10).copy()

# Aplica formatação de milhar com ponto e remove casas decimais
df_exibicao['total_leads'] = df_exibicao['total_leads'].apply(
    lambda x: f"{x:,.0f}".replace(",", "_").replace(".", ",").replace("_", ".")
)
# Aplica formatação de porcentagem com duas casas decimais
df_exibicao['participacao_%'] = df_exibicao['participacao_%'].apply(
    lambda x: f"{x:.2%}"
)

# Renomeia o index 
df_exibicao.index = df_volume_canal.index[:10]

print("--- Diagnóstico: Ranking de Volume de Leads por utm_source ---")
# Usei display para uma saída formatada e limpa no notebook
display(df_exibicao)

--- Diagnóstico: Ranking de Volume de Leads por utm_source ---


,canal_aquisicao,total_leads,participacao_%
0,google,19.621,37.66%
1,instagram,17.539,33.67%
2,whatsapp oficial,4.574,8.78%
3,desconhecido,3.766,7.23%
4,não informado,2.066,3.97%
5,typeform-blip,1.187,2.28%
6,typeform-ex membro,593,1.14%
7,typeform-indicação interna,500,0.96%
8,typeform-outros,446,0.86%
9,typeform-gentileza,425,0.82%


O canal de aquisição gerou o maior volume de leads foi o google com 37.66%

PERGUNTA 05: Considere que a Base de Dados disponibilizada está no BigQuery no esquema revenue e a tabelas chamam leads, visitas e contratos. Escreva abaixo uma query que retorne, o mês de criação do lead, a quantidade de leads criados, a conversão da safra em venda e a quantidade de leads que visitam as unidades ao menos 1 vez. O resultado deve estar ordenado do mês mais recente para o mês mais antigo e ter apenas os dados referentes ao segundo semestre de 2024.

In [ ]:
import pandas as pd
import pandasql
from pathlib import Path
from IPython.display import display

# 1. CRIAR PASTA E ARQUIVO SQL
# Define o caminho para a pasta e o arquivo de query
PATH_QUERIES = Path("./queries")
PATH_QUERIES.mkdir(exist_ok=True)
SQL_FILE_PATH = PATH_QUERIES / "analise_safra.sql"

# A lógica de junção é por MÊS, contornando a falta de lead_id na tabela visitas.
query_sql_agregada = """
SELECT
    t1.mes_safra,
    t1.total_leads_criados,
    t1.leads_vendidos,
    -- Conversão da Safra em Venda
    CAST(t1.leads_vendidos AS REAL) / t1.total_leads_criados AS taxa_conversao_venda,
    -- Quantidade de Visitas Realizadas (Agregadas por Mês)
    COALESCE(t2.visitas_realizadas, 0) AS visitas_realizadas
FROM
    (
        -- Subquery 1 (t1): Métricas de Volume e Venda por Mês de Safra
        SELECT
            strftime('%Y-%m', data_criacao) AS mes_safra,
            COUNT(DISTINCT lead_id) AS total_leads_criados,
            COUNT(DISTINCT CASE WHEN data_venda IS NOT NULL THEN lead_id END) AS leads_vendidos
        FROM
            df_funil
        WHERE
            -- Filtro para o 2º semestre de 2024
            data_criacao >= '2024-07-01' AND data_criacao < '2025-01-01'
        GROUP BY 1
    ) AS t1
LEFT JOIN
    (
        -- Subquery 2 (t2): Visitas Únicas por Mês de Referência
        SELECT
            mes_referencia AS mes_safra,
            COUNT(DISTINCT id_visita) AS visitas_realizadas
        FROM
            visitas
        WHERE
            data_visita >= '2024-07-01' AND data_visita < '2025-01-01'
        GROUP BY 1
    ) AS t2 ON t1.mes_safra = t2.mes_safra

ORDER BY
    t1.mes_safra DESC;
"""

# Salva a string SQL no arquivo
with open(SQL_FILE_PATH, "w") as f:
    f.write(query_sql_agregada)

print(f"✅ Arquivo de query salvo em: {SQL_FILE_PATH}")

# --- 2. Carregamento e Preparação dos DataFrames (Data Mart) ---
# Necessário para que as tabelas (df_funil, visitas) existam na memória para o pandasql.

base_path = Path("../processed")

# Leads
try:
    lead = pd.read_csv(base_path / "leads_limpo.csv")
except FileNotFoundError:
    print(f"⚠️ Erro: Não foi possível carregar 'leads_limpo.csv' do caminho: {base_path / 'leads_limpo.csv'}. Ajuste 'base_path'.")
    raise # Interrompe a execução para que o usuário corrija o path

date_cols = ["data_criacao", "data_perda", "data_venda"]
for col in date_cols:
    lead[col] = pd.to_datetime(lead[col], errors="coerce")

lead["flag_venda"] = lead["data_venda"].notna().astype(int)
lead["mes_safra"] = lead["data_criacao"].dt.to_period("M").astype(str)
df_funil = lead[["lead_id", "data_criacao", "mes_safra", "data_venda", "flag_venda"]].copy()

# Visitas
try:
    visitas = pd.read_csv(base_path / "visitas_limpo.csv")
except FileNotFoundError:
    print(f"⚠️ Erro: Não foi possível carregar 'visitas_limpo.csv' do caminho: {base_path / 'visitas_limpo.csv'}. Ajuste 'base_path'.")
    raise # Interrompe a execução para que o usuário corrija o path

visitas["data_visita"] = pd.to_datetime(visitas["data_visita"], errors="coerce")
visitas = visitas.dropna(subset=["data_visita"])
visitas["mes_referencia"] = visitas["data_visita"].dt.to_period("M").astype(str)
visitas = visitas[["id_visita", "data_visita", "mes_referencia"]].copy()

print("✅ DataFrames 'df_funil' e 'visitas' carregados e prontos para o pandasql.")

✅ Arquivo de query salvo em: queries\analise_safra.sql
✅ DataFrames 'df_funil' e 'visitas' carregados e prontos para o pandasql.


In [16]:
# ... (Continuação do Bloco 1)

# Define o Data Mart 'df_funil' (tabela 'leads' para o SQL)
df_funil = lead[["lead_id", "data_criacao", "mes_safra", "data_venda", "flag_venda"]].copy()

# 💥 CORREÇÃO: Reseta o índice para remover qualquer coluna de índice extra
df_funil = df_funil.reset_index(drop=True)


# --- 3. Carregamento e Preparação do VISITAS (Tabela 'visitas') ---
visitas = pd.read_csv(base_path / "visitas_limpo.csv")
# ... (restante da preparação de 'visitas')

# Tabela 'visitas' para o SQL
visitas = visitas[["id_visita", "data_visita", "mes_referencia"]].copy()

# 💥 CORREÇÃO: Reseta o índice para remover qualquer coluna de índice extra
visitas = visitas.reset_index(drop=True)

print("✅ DataFrames 'df_funil' e 'visitas' carregados com índice resetado, prontos para o pandasql.")

✅ DataFrames 'df_funil' e 'visitas' carregados com índice resetado, prontos para o pandasql.


In [18]:
from IPython.display import display
from pathlib import Path
import pandasql


# ===============================================================
# 3. EXECUÇÃO DA QUERY E EXIBIÇÃO
# ===============================================================

# Lendo o conteúdo do arquivo SQL
try:
    SQL_FILE_PATH = Path("./queries") / "analise_safra.sql"
    with open(SQL_FILE_PATH, "r") as f:
        query_executar = f.read()
except FileNotFoundError:
    print(f"⚠️ Erro: Arquivo SQL não encontrado em: {SQL_FILE_PATH}.")
    raise

# Execução da Query
df_resultado_safra_sql = pandasql.sqldf(query_executar, globals())

print("--- Análise de Safra via pandasql (Resultado Final) ---")

#  Usar .rename() para renomear apenas as colunas de exibição
df_final_display = df_resultado_safra_sql.rename(columns={
    'mes_safra': 'Mês de Safra',
    'total_leads_criados': 'Leads Criados',
    'leads_vendidos': 'Leads Vendidos',
    'taxa_conversao_venda': 'Taxa Conversão Venda',
    'visitas_realizadas': 'Visitas Agregadas'
})

# Tratamento de colunas numéricas
df_final_display['Visitas Agregadas'] = df_final_display['Visitas Agregadas'].fillna(0).astype(int)
df_final_display['Taxa Conversão Venda'] = df_final_display['Taxa Conversão Venda'].apply(lambda x: f"{x:.2%}")

# Formatação de números grandes
for col in ['Leads Criados', 'Leads Vendidos', 'Visitas Agregadas']:
    
    # Garantir que a coluna é numérica (necessário pelo pandasql)
    df_final_display[col] = pd.to_numeric(df_final_display[col], errors='coerce').fillna(0).astype(int)
    
    # Aplicar formatação para o padrão brasileiro
    df_final_display[col] = df_final_display[col].apply(lambda x: "{:,.0f}".format(x).replace(",", "_").replace(".", ",").replace("_", "."))

display(df_final_display)

print("✅ Análise concluída")

--- Análise de Safra via pandasql (Resultado Final) ---


,Mês de Safra,Leads Criados,Leads Vendidos,Taxa Conversão Venda,Visitas Agregadas
0,2024-12,2.806,266,9.48%,420
1,2024-11,3.654,421,11.52%,530
2,2024-10,3.872,505,13.04%,674
3,2024-09,3.853,509,13.21%,667
4,2024-08,3.668,475,12.95%,699
5,2024-07,3.717,506,13.61%,750


✅ Análise concluída
